In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


1. 자신의 주변에 위치한 도서관 파악하기

1-1 사용자의 위치 정보 파악하기

- 사용자의 위치는 GPS나 IP 주소를 기반으로 파악할 수 있습니다.
- Python에서는 geopy 라이브러리를 이용해 위치 정보를 다룰 수 있습니다.
- 예를 들어, 사용자의 IP를 이용해 대략적인 위치를 파악할 수 있습니다.

In [3]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time

def get_user_location(address, retries=3, timeout=10):
    geolocator = Nominatim(user_agent="library_finder")
    for i in range(retries):
        try:
            user_location = geolocator.geocode(address, timeout=timeout)
            if user_location:
                return (user_location.latitude, user_location.longitude)
            else:
                print(f"Unable to geocode address: {address}")
                return None
        except (GeocoderTimedOut, GeocoderUnavailable) as e:
            print(f"Geocoding attempt {i+1} failed: {e}")
            time.sleep(2)  # Wait before retrying
    print("Max retries exceeded. Geocoding failed.")
    return None

address = input('주소나 위치 정보를 입력하세요.')  # 사용자의 주소나 위치 정보
user_location = get_user_location(address)
if user_location:
    latitude, longitude = user_location
    print(f"User's location: {latitude}, {longitude}")
else:
    print("Failed to get the user's location.")

User's location: 35.5149524, 129.2938637


1.2 도서관 위치 정보 가져오기

- 지역별 도서관의 위치 데이터를 저장하거나, 공공 API를 이용해 도서관 위치를 받아옵니다. 한국의 경우, 공공데이터 포털에서 도서관 위치와 관련된 데이터를 제공할 수 있습니다.
- geopy.distance 모듈을 사용하여 두 지점 사이의 직선 거리를 계산할 수 있습니다.

In [4]:
import pandas as pd
from geopy.distance import geodesic
import math

# CSV 파일에서 도서관 위치 데이터 읽어오기
def load_libraries(csv_file):
    df = pd.read_csv(csv_file, encoding='UTF8')
    libraries = []
    
    for index, row in df.iterrows():
        # NaN 값이 포함된 좌표는 무시
        if pd.isna(row['LBRRY_LA']) or pd.isna(row['LBRRY_LO']):
            continue
        
        # 위도와 경도 값이 유효한지 확인
        try:
            lat = float(row['LBRRY_LA'])
            lon = float(row['LBRRY_LO'])
            if not math.isfinite(lat) or not math.isfinite(lon):
                continue
        except ValueError:
            continue

        #libraries = row['LBRRY_NM'], lat, lon

        libraries.append({
            'name': row['LBRRY_NM'],
            'coords': (lat, lon)
        })

    return libraries

# 사용자의 위치와 도서관 간의 거리를 계산
def find_nearest_library(user_coords, libraries):
    nearest_library = None
    shortest_distance = float('inf')
    
    for library in libraries:
        distance = geodesic(user_coords, library['coords']).km
        if distance < shortest_distance:
            shortest_distance = distance
            nearest_library = library
    
    return nearest_library, shortest_distance

# 예제 사용
csv_file = 'C:/개인 프로젝트/boothcamp/도서찾기 프로젝트/도서관 정보_202407.csv'  # CSV 파일 경로
user_coords = (latitude, longitude)  # 사용자의 위도와 경도

libraries = load_libraries(csv_file)
nearest_library, distance = find_nearest_library(user_coords, libraries)

if nearest_library:
    print(f"가장 가까운 도서관은 {nearest_library['name']}이며, 거리는 {distance:.2f} km 입니다.")
else:
    print("주변에 도서관이 없습니다.")


가장 가까운 도서관은 울산남부도서관이며, 거리는 2.32 km 입니다.


1.3 도서관까지의 보행 거리 계산

- 보행 거리나 실제 경로 거리를 계산하려면 Google Maps API나 OpenStreetMap의 osmnx 같은 라이브러리를 사용할 수 있습니다.
- osmnx는 도로 네트워크를 기반으로 경로를 계산하는데 유용합니다.

In [5]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import osmnx as ox
import networkx as nx

def calculate_walking_distance(user_coords, library_coords):
    # 사용자의 위치 주변의 도보 가능한 네트워크 그래프 생성
    G = ox.graph_from_point(user_coords, dist=5000, network_type='walk')

    # 그래프를 UTM으로 투영
    G_proj = ox.project_graph(G)

    # 사용자의 위치와 도서관의 위치에서 가장 가까운 노드를 찾기
    orig_node = ox.distance.nearest_nodes(G_proj, user_coords[1], user_coords[0])
    dest_node = ox.distance.nearest_nodes(G_proj, library_coords[1], library_coords[0])
    print(orig_node)
    print(dest_node)

    # 최단 경로 및 거리를 계산
    route = nx.shortest_path(G_proj, orig_node, dest_node, weight='length')
    distance = nx.shortest_path_length(G_proj, orig_node, dest_node, weight='length')

    return distance / 1000 # meters to kilometers

# 사용자의 위치 예시
user_coords = (latitude, longitude)  # 위도, 경도

# 도서관의 위치 예시
#library_coords = (lat, lon)  # 위도, 경도 (예시 도서관 위치)
library_coords = libraries[0]['coords']
print(user_coords)
print(library_coords)
walking_distance = calculate_walking_distance(user_coords, library_coords)
print(f"도서관까지 도보 거리 : {walking_distance:.2f} km")


(35.5150738, 129.2940681)
(37.9053026, 127.206376)


c:\Users\qq\AppData\Local\Programs\Python\Python310\lib\site-packages\osmnx\graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


436417102
436417102
도서관까지 도보 거리 : 0.00 km


In [7]:
import osmnx as ox
import networkx as nx

def calculate_walking_distance(user_coords, library_coords):
    # 사용자의 위치 주변의 도보 가능한 네트워크 그래프 생성
    G = ox.graph_from_point(user_coords, dist=5000, network_type='walk')

    # 그래프를 UTM으로 투영
    G_proj = ox.project_graph(G)

    # 사용자의 위치와 도서관의 위치에서 가장 가까운 노드를 찾기
    orig_node = ox.distance.nearest_nodes(G_proj, user_coords[0], user_coords[1])  # 위도, 경도 순서
    dest_node = ox.distance.nearest_nodes(G_proj, library_coords[0], library_coords[1])  # 위도, 경도 순서
    print(f"Original node: {orig_node}")
    print(f"Destination node: {dest_node}")

    # 최단 경로 및 거리를 계산
    try:
        route = nx.shortest_path(G_proj, orig_node, dest_node, weight='length')
        distance = nx.shortest_path_length(G_proj, orig_node, dest_node, weight='length')
    except nx.NetworkXNoPath:
        print("두 지점 간에 경로를 찾을 수 없습니다.")
        return float('inf')

    return distance  # meters

# 사용자의 위치 예시
user_coords = (latitude,longitude)  # 위도, 경도

# 도서관의 위치 예시
library_coords = libraries[0]['coords']  # 위도, 경도
print(f"User coordinates: {user_coords}")
print(f"Library coordinates: {library_coords}")

walking_distance = calculate_walking_distance(user_coords, library_coords)
print(f"도서관까지 도보 거리: {walking_distance:.2f} meters")


User coordinates: (35.5149524, 129.2938637)
Library coordinates: (35.535816, 129.2951888)


c:\Users\qq\AppData\Local\Programs\Python\Python310\lib\site-packages\osmnx\graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


Original node: 436417102
Destination node: 436417102
도서관까지 도보 거리: 0.00 meters


In [13]:
import requests

def get_walking_distance_osrm(start_coords, end_coords):
    base_url = 'http://router.project-osrm.org/route/v1/walking/'
    coordinates = f"{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}"
    url = f"{base_url}{coordinates}?overview=false"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['routes']:
            distance = data['routes'][0]['distance'] / 1000  # Convert meters to kilometers
            duration = data['routes'][0]['duration'] / 60  # Convert seconds to minutes
            
            # Optional: Apply a speed adjustment factor
            speed_adjustment_factor = 1.2  # Adjust this factor based on the user's walking speed
            duration_adjusted = duration * speed_adjustment_factor
            
            return distance, duration_adjusted
        else:
            return "No routes found.", None
    else:
        return "Request failed.", None

# 예제 사용
user_coords = (latitude, longitude)  # 사용자의 위도와 경도
library_coords = nearest_library['coords']  # 가장 가까운 도서관의 위도와 경도

distance, duration = get_walking_distance_osrm(user_coords, library_coords)
if distance is not None:
    print(f"가장 가까운 도서관까지의 보행 거리는 {distance:.2f} km이며, 소요 시간은 {duration:.2f} 분입니다.")
else:
    print("보행 경로를 찾을 수 없습니다.")

가장 가까운 도서관까지의 보행 거리는 5.23 km이며, 소요 시간은 8.93 분입니다.


In [1]:
import requests
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
from geopy.distance import geodesic
import pandas as pd
import math
import time

# 사용자 위치를 가져오는 함수
def get_user_location(address, retries=3, timeout=10):
    geolocator = Nominatim(user_agent="library_finder")
    for i in range(retries):
        try:
            user_location = geolocator.geocode(address, timeout=timeout)
            if user_location:
                return (user_location.latitude, user_location.longitude)
            else:
                print(f"Unable to geocode address: {address}")
                return None
        except (GeocoderTimedOut, GeocoderUnavailable) as e:
            print(f"Geocoding attempt {i+1} failed: {e}")
            time.sleep(2)  # Wait before retrying
    print("Max retries exceeded. Geocoding failed.")
    return None

# CSV 파일에서 도서관 위치 데이터 읽어오기
def load_libraries(csv_file):
    df = pd.read_csv(csv_file, encoding='UTF8')
    libraries = []
    
    for index, row in df.iterrows():
        if pd.isna(row['LBRRY_LA']) or pd.isna(row['LBRRY_LO']):
            continue
        
        try:
            lat = float(row['LBRRY_LA'])
            lon = float(row['LBRRY_LO'])
            if not math.isfinite(lat) or not math.isfinite(lon):
                continue
        except ValueError:
            continue

        libraries.append({
            'name': row['LBRRY_NM'],
            'coords': (lat, lon)
        })

    return libraries

# 사용자의 위치와 도서관 간의 거리를 계산
def find_nearest_library(user_coords, libraries):
    nearest_library = None
    shortest_distance = float('inf')
    
    for library in libraries:
        distance = geodesic(user_coords, library['coords']).km
        if distance < shortest_distance:
            shortest_distance = distance
            nearest_library = library
    
    return nearest_library, shortest_distance

# OSRM을 이용한 보행 거리 및 시간 계산
def get_walking_distance_osrm(start_coords, end_coords):
    base_url = 'http://router.project-osrm.org/route/v1/walking/'
    coordinates = f"{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}"
    url = f"{base_url}{coordinates}?overview=false"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['routes']:
            distance = data['routes'][0]['distance'] / 1000  # 미터를 킬로미터로 변환
            duration = data['routes'][0]['duration'] / 60  # 초를 분으로 변환
            
            # 현실적인 예상 소요 확인
            average_walking_speed_kmph = 5  # 평균 보행 속도 (km/h)
            expected_duration = distance / average_walking_speed_kmph * 60  # 분 단위로 계산
            
            # 선택 사항: 속도 조정 인자 적용 (필요한 경우)
            speed_adjustment_factor = 1.2  # 사용자의 보행 속도에 따라 이 인자를 조정
            duration_adjusted = duration * speed_adjustment_factor
            
            # OSRM의 소요 시간이 너무 짧을 경우, 현실적인 소요 시간을 사용
            if duration_adjusted < expected_duration * 0.5:
                duration_adjusted = expected_duration
            
            return distance, duration_adjusted
        else:
            return "No routes found.", None
    else:
        return "Request failed.", None

# 사용자 입력 처리
address = input('주소나 위치 정보를 입력하세요: ')  # 사용자의 주소나 위치 정보
user_location = get_user_location(address)
if user_location:
    latitude, longitude = user_location
    print(f"User's location: {latitude}, {longitude}")
else:
    print("Failed to get the user's location.")
    exit()

# CSV 파일 경로 설정 및 도서관 데이터 로드
csv_file = 'C:/개인 프로젝트/boothcamp/도서찾기 프로젝트/도서관 정보_202407.csv'  # CSV 파일 경로
libraries = load_libraries(csv_file)

# 가장 가까운 도서관 찾기
nearest_library, distance = find_nearest_library(user_location, libraries)

if nearest_library:
    print(f"가장 가까운 도서관은 {nearest_library['name']}이며, 거리는 {distance:.2f} km 입니다.")
    
    # 보행 거리 및 시간 계산
    library_coords = nearest_library['coords']
    walking_distance, walking_duration = get_walking_distance_osrm(user_location, library_coords)
    
    if walking_distance is not None:
        print(f"가장 가까운 도서관까지의 보행 거리는 {walking_distance:.2f} km이며, 소요 시간은 {walking_duration:.2f} 분입니다.")
    else:
        print("보행 경로를 찾을 수 없습니다.")
else:
    print("주변에 도서관이 없습니다.")

User's location: 37.553468699999996, 127.02377989112765
가장 가까운 도서관은 성동구립 금호도서관이며, 거리는 0.12 km 입니다.
가장 가까운 도서관까지의 보행 거리는 0.38 km이며, 소요 시간은 4.56 분입니다.


2. 도서관에서 본인이 원하는 책이 있는지 여부 확인

2.1 도서관 책 목록 가져오기

- 각 도서관의 책 목록을 제공하는 API가 있다면 이를 활용해 책 목록을 받아옵니다.
- 예를 들어, 특정 도서관의 웹사이트에서 제공하는 Open API를 사용할 수 있습니다.

In [20]:
import requests
import xml.etree.ElementTree as ET

def check_book_in_library(library_api_url, book_title):
    
    params = {
        #'serviceKey': 'bNwH7fMESNzwAdqKn4rn3mvLVwSHVwt%2Bmze%2Fu1HudJZlAtB0KitYcJHsQn15UdGWTFrSW%2BMmpNd7VxdUTWfLCw%3D%3D',
        'serviceKey': '377f5a2ddbf1d16afa69eba43a18713050e5c3e4ad7f85c05f2182f548df796d',    # 반드시 API 키를 여기에 넣으세요
        'title': book_title
    }
    response = requests.get(library_api_url, params=params)
    
    if response.status_code == 200:
        # XML 데이터를 파싱
        root = ET.fromstring(response.content)
        
        # 예제에 따라 도서의 유무를 판단하는 부분이 다를 수 있습니다.
        # 여기서는 예제로 첫 번째 도서의 상태를 가져오는 방법을 설명합니다.
        for item in root.findall('.//item'):
            book_name = item.find('title').text
            if book_name == book_title:
                available = item.find('loanAvailable').text  # 대출 가능 여부 필드 예시
                return available == 'Y'  # 대출 가능이면 True 반환
        
        return False
    else:
        print("Error: API 요청이 실패했습니다.")
        return False

# 예제 사용
library_api_url = "https://www.nl.go.kr/NL/search/openApi/search.do?key=[377f5a2ddbf1d16afa69eba43a18713050e5c3e4ad7f85c05f2182f548df796d]&apiType=xml&srchTarget=total&kwd=%ED%86%A0%EC%A7%80&pageSize=10&pageNum=1&sort=&category=%EB%8F%84%EC%84%9C"
book_title = ""
is_available = check_book_in_library(library_api_url, book_title)
print(f"Is the book available? {'Yes' if is_available else 'No'}")


Is the book available? No


In [17]:
import csv

def check_book_in_library(csv_file_path, book_title):
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['TITLE_NM'].strip().lower() == book_title.strip().lower():
                return row['Availability'].strip().lower() == 'yes'
    return False

csv_file_path = '공공도서관 소장도서_202407-1.csv'  # CSV 파일 경로
book_title = input('찾고자 하는 책의 제목을 입력하세요.')  # 찾고자 하는 책의 제목

is_available = check_book_in_library(csv_file_path, book_title)
print(f"Is the book '{book_title}' available? {'Yes' if is_available else 'No'}")

KeyError: 'Availability'

2.2 원하는 책의 유무 확인

- 사용자가 입력한 책 제목을 검색해 각 도서관의 API에서 해당 책이 있는지 확인합니다. 여러 도서관에 대해 반복해서 검색할 수 있습니다.

In [ ]:
libraries = [
    {"name": "Library 1", "api_url": "http://example.com/library1_api"},
    {"name": "Library 2", "api_url": "http://example.com/library2_api"},
    # more libraries...
]

for library in libraries:
    is_available = check_book_in_library(library['api_url'], book_title)
    if is_available:
        print(f"{book_title} is available at {library['name']}")
    else:
        print(f"{book_title} is not available at {library['name']}")


최종적으로 애플리케이션을 웹 애플리케이션으로 확장하려면
- 웹 프레임워크: Flask나 Django를 사용하여 API와 사용자 인터페이스를 구축할 수 있습니다.
- 지도 API: Google Maps, OpenStreetMap 등을 이용해 도서관 위치를 지도에 표시하고, 경로를 안내할 수 있습니다.
-  프론트엔드: HTML, CSS, JavaScript로 사용자 인터페이스를 구성합니다.

추가 고려 사항
- 데이터베이스: 도서관 위치와 책 목록을 데이터베이스에 저장하고 관리할 수 있습니다. SQLite, PostgreSQL, MySQL 등을 사용할 수 있습니다.
- 사용자 인증: 사용자가 개인 계정을 통해 특정 도서관을 즐겨찾기하거나 책을 예약할 수 있도록 하는 기능을 추가할 수 있습니다.

위의 구조를 기반으로 프로젝트를 시작하면, 단계별로 구현해나갈 수 있을 것입니다. 처음에는 간단한 콘솔 애플리케이션으로 시작한 뒤, 점차 웹 애플리케이션으로 확장해 나가면 됩니다.